# Paralogs genes

### 1.1 Including libraries 

In [1]:
from Bio.Blast import NCBIWWW
from Bio.Blast import Applications
from Bio.Blast.Applications import NcbiblastxCommandline
import gzip 
import io
import math
import matplotlib.pyplot as plt
from ftplib import FTP
import os
import numpy as np
import pandas as pd
import patoolib
import random
import re
import seaborn as sns
import scipy as sc
import string
from tqdm import tqdm
import datetime
import xml.etree.ElementTree as ET
%matplotlib inline

### 1.2 Data for Blastp

In [2]:
def data_for_blastp(bacterium):
    # diving to database
    ftp = FTP('ftp.ncbi.nlm.nih.gov')
    ftp.login()
    ftp.cwd('genomes/refseq/bacteria')
    ftp.cwd(bacterium)
    
    domain_list = ['representative','reference','latest_assembly_versions']
    for domain in domain_list:
        if domain in ftp.nlst():
            ftp.cwd(domain)
            break
            
    current_strain = ftp.nlst()
    ftp.cwd(current_strain[0])
     
    namefile = current_strain[0] + '_protein.faa.gz' # file, which i want
    out = "C://idea_projects_my//my_file.gz" # place on my disk
    
    with open(out,'wb') as f:
        ftp.retrbinary('RETR ' + namefile,f.write)
        f_read = gzip.open(out,'rb')
    
    os.mkdir('C:\\idea_projects_my\\scherichia')
    patoolib.extract_archive(out,outdir = 'C:\\idea_projects_my\\scherichia')  
    os.listdir('C:\\idea_projects_my\\scherichia')
    new_out = 'C:\\idea_projects_my\\scherichia'
    for unarchived_file in os.listdir('C:\\idea_projects_my\\scherichia'):
        with open('C:\\idea_projects_my\\scherichia' + '\\' + unarchived_file, 'r') as myfile:
            data = myfile.read()
    #remove directories       
    os.remove('C:\\idea_projects_my\\scherichia\\my_file')
    os.rmdir('C:\\idea_projects_my\\scherichia')
    
    
    return data

### 1.3 Splitting data to genes

In [3]:
def splitting(data):
    names_of_genes = data.split(sep = '>WP_')
    
    return names_of_genes

### 1.4 XML_parser for data output of Blastp

In [65]:
def Parse_XML_and_holder(xml_current_file,gene_1):
    name_of_gene,length,identity,e_value,gen = list(),list(),list(),list(),list()
    with open('docu.xml','w') as file:
        file.write(str(xml_current_file))
        
    root = ET.parse('docu.xml').getroot()
    for subtags in root.iter('Iteration_hits'):
        for subsubtags in subtags:
            for objects in subsubtags:
                if(objects.tag == 'Hit_def'):
                    name_of_gene_cur = objects.text
                if(objects.tag == 'Hit_len'):
                    length_cur = int(objects.text)
                if(objects.tag == 'Hit_hsps'):
                    for tags in objects[0]:
                        if(tags.tag == 'Hsp_evalue'):
                            e_value_cur = float(tags.text)
                        if(tags.tag == 'Hsp_identity'):
                            identity_cur = int(tags.text)
                            if(identity_cur/length_cur > 0.5 and identity_cur/length_cur != 1):
                                gen.append(gene_1)
                                name_of_gene.append(name_of_gene_cur)
                                length.append(length_cur)
                                e_value.append(e_value_cur)
                                identity.append(float(identity_cur/length_cur))
                        
    return name_of_gene,length,identity,e_value,gen


### 1.5 BlastP

In [66]:
def base_program(names_of_genes,bacterium):
    
    gen,gene_2,length,identity,e_value,bact = list(),list(),list(),list(),list(),list()
    #ways for genes
    out_sample = 'C://idea_projects_my//Escherichia_sample'
    out_database = 'C://idea_projects_my//Escherichia_database'
    
    for i, sample in tqdm(enumerate(names_of_genes)):
        database = names_of_genes[i:]
        try:
            with open(out_sample,'w')as file:
                file.write(str(sample))
            with open(out_database,'w')as file:
                file.write("".join(map(lambda x: ">WP_" + x, database)))
        except(FileNotFoundError,OSError):
            print('problems with directories')
            continue
        
        gene_1 = "WP_" + str(sample)[: str(sample).find(" ")]
        
        !makeblastdb -in C://idea_projects_my//Escherichia_database -dbtype prot -out C://idea_projects_my//Escherichia_database      
        
        result = NcbiblastxCommandline(cmd='blastp',num_threads = 4,query= out_sample,
                              db=out_database,evalue=1e-9,outfmt = 5)
        
        try:
            #out, err = result()
            output_of_xml = Parse_XML_and_holder(result()[0],gene_1)
            gene_2.extend(output_of_xml[0])
            length.extend(output_of_xml[1])
            identity.extend(output_of_xml[2])
            e_value.extend(output_of_xml[3])
            gen.extend(output_of_xml[4])
            
            
        except:
            print("oops" + str(i))
            continue
            
        
    return gene_2,length,identity,e_value,bacterium,gen


### 1.7 Result of BlastP

In [106]:
data = data_for_blastp('Abiotrophia_defectiva')
names_of_genes = splitting(data)
final = base_program(names_of_genes,'Abiotrophia_defectiva')

0it [00:00, ?it/s]

patool: Extracting C://idea_projects_my//my_file.gz ...
patool: running "C:\Program Files\7-Zip\7z.EXE" e -oC:\idea_projects_my\scherichia -- C://idea_projects_my//my_file.gz
patool: ... C://idea_projects_my//my_file.gz extracted to `C:\idea_projects_my\scherichia'.


Building a new DB, current time: 01/09/2020 14:34:39
New DB name:   C:\\idea_projects_my\Escherichia_database
New DB title:  C://idea_projects_my//Escherichia_database
Sequence type: Protein
Deleted existing Protein BLAST database named C:\\idea_projects_my\Escherichia_database
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 1816 sequences in 0.132171 seconds.
oops0


1it [00:00,  3.29it/s]



Building a new DB, current time: 01/09/2020 14:34:39
New DB name:   C:\\idea_projects_my\Escherichia_database
New DB title:  C://idea_projects_my//Escherichia_database
Sequence type: Protein
Deleted existing Protein BLAST database named C:\\idea_projects_my\Escherichia_database
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 1816 sequences in 0.157954 seconds.


2it [00:00,  3.02it/s]

...

1815it [11:10,  4.18it/s]



Building a new DB, current time: 01/09/2020 14:45:49
New DB name:   C:\\idea_projects_my\Escherichia_database
New DB title:  C://idea_projects_my//Escherichia_database
Sequence type: Protein
Deleted existing Protein BLAST database named C:\\idea_projects_my\Escherichia_database
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 2 sequences in 0.00272711 seconds.


1816it [11:10,  4.03it/s]



Building a new DB, current time: 01/09/2020 14:45:50
New DB name:   C:\\idea_projects_my\Escherichia_database
New DB title:  C://idea_projects_my//Escherichia_database
Sequence type: Protein
Deleted existing Protein BLAST database named C:\\idea_projects_my\Escherichia_database
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 1 sequences in 0.00140844 seconds.


1817it [11:10,  4.15it/s]


### 1.8 Creating database

In [107]:
data = {"bact":final[4],"gene_1":final[5],"gene_2":final[0],"length_2":final[1],"identity":final[2],"e_value":final[3]}
df = pd.DataFrame(data)
df.tail()

,bact,gene_1,gene_2,length_2,identity,e_value
45,Abiotrophia_defectiva,WP_023392501.1,WP_035364214.1 MgtC/SapB family protein [Abiot...,225,0.697778,1.429400e-117
46,Abiotrophia_defectiva,WP_035364090.1,WP_156905306.1 nucleotide pyrophosphohydrolase...,95,0.768421,6.302600e-52
47,Abiotrophia_defectiva,WP_035364527.1,WP_143227653.1 MULTISPECIES: ATP-binding casse...,245,0.600000,4.019540e-106
48,Abiotrophia_defectiva,WP_084796473.1,WP_156905290.1 hypothetical protein [Abiotroph...,70,0.914286,4.099850e-45
49,Abiotrophia_defectiva,WP_084796515.1,WP_156905292.1 CHAP domain-containing protein ...,219,0.561644,9.190670e-82


In [108]:
df['gene_2'] = df['gene_2'].apply(lambda x: str(x)[:str(x).find(" ")])

In [111]:
df.tail()

,bact,gene_1,gene_2,length_2,identity,e_value
45,Abiotrophia_defectiva,WP_023392501.1,WP_035364214.1,225,0.697778,1.429400e-117
46,Abiotrophia_defectiva,WP_035364090.1,WP_156905306.1,95,0.768421,6.302600e-52
47,Abiotrophia_defectiva,WP_035364527.1,WP_143227653.1,245,0.600000,4.019540e-106
48,Abiotrophia_defectiva,WP_084796473.1,WP_156905290.1,70,0.914286,4.099850e-45
49,Abiotrophia_defectiva,WP_084796515.1,WP_156905292.1,219,0.561644,9.190670e-82


### 2. Finding placement of genes

In [112]:
def data_for_placements(bacterium):
    # diving to database
    ftp = FTP('ftp.ncbi.nlm.nih.gov')
    ftp.login()
    ftp.cwd('genomes/refseq/bacteria')
    ftp.cwd(bacterium)
    
    domain_list = ['representative','reference','latest_assembly_versions']
    for domain in domain_list:
        if domain in ftp.nlst():
            ftp.cwd(domain)
            break
            
    current_strain = ftp.nlst()
    ftp.cwd(current_strain[0])
     
    namefile = current_strain[0] + '_translated_cds.faa.gz' # file, which i want
    out = "C://idea_projects_my//my_file.gz" # place on my disk
    
    with open(out,'wb') as f:
        ftp.retrbinary('RETR ' + namefile,f.write)
        f_read = gzip.open(out,'rb')
    
    os.mkdir('C:\\idea_projects_my\\scherichia')
    patoolib.extract_archive(out,outdir = 'C:\\idea_projects_my\\scherichia')  
    os.listdir('C:\\idea_projects_my\\scherichia')
    new_out = 'C:\\idea_projects_my\\scherichia'
    for unarchived_file in os.listdir('C:\\idea_projects_my\\scherichia'):
        with open('C:\\idea_projects_my\\scherichia' + '\\' + unarchived_file, 'r') as myfile:
            data = myfile.read()
    #remove directories       
    os.remove('C:\\idea_projects_my\\scherichia\\my_file')
    os.rmdir('C:\\idea_projects_my\\scherichia')
    
    
    return data

In [117]:
def splitting(data):
    genes = data.split(sep = '>')
    genes = genes[1:]
    return genes

In [121]:
def information_of_gene(gene,genes):
    for element in genes:
        if(gene in element):
            return element

In [ ]:
def location_of_gene(element):
    return re.findall(r)

In [122]:
data = data_for_placements('Abiotrophia_defectiva')
res = splitting(data)
query = information_of_gene('WP_035364214.1',res)
query

patool: Extracting C://idea_projects_my//my_file.gz ...
patool: running "C:\Program Files\7-Zip\7z.EXE" e -oC:\idea_projects_my\scherichia -- C://idea_projects_my//my_file.gz
patool: ... C://idea_projects_my//my_file.gz extracted to `C:\idea_projects_my\scherichia'.


'lcl|NZ_KI535340.1_prot_WP_035364214.1_1203 [locus_tag=GCWU000182_RS06100] [protein=MgtC/SapB family protein] [protein_id=WP_035364214.1] [location=1311627..1312304] [gbkey=CDS]\nMSAYVILVRLCLAILFAGVIGYDRQRKNRPAGLRTHILVCIGATAISMIQAAVFYQKINEYPQISVDQVRLLAPIVSGIG\nFLGAGTIVVTKQRVAGLTTAASLWTTAALGIALGMGYYEIAGLTFLGVIFSLTVIRFIVRIPDVKRLEIHYVHRVATKEF\nFADFFKTHHIELTDVQFDVTNVDGKKIYRNTFTIHLPKDLTHVQLIEELSTNPDIMKVRLVSINE\n'

In [130]:
re.findall(r'location=\w+',query)

['location=1311627']

## 3. Origin Terminator